In [6]:
# test cifar, imagenet acc

from Dataloaders import dataloader_image_20
train, test, val, _ = dataloader_image_20.Dataloader_imagenet_20_integrated()

from Models import mobilenetv2, resnet
# client, server = mobilenetv2.mobilenetv2_splitter(num_classes=1000, weight_root='./Weights/imagenet/')
client, server = resnet.resnet_splitter(num_classes=1000, weight_root='./Weights/imagenet/', layers=50)

client = client.cuda()
server = server.cuda()
client.eval()
server.eval()

import torch
from tqdm import tqdm

# calculate the correct rate of each label
with torch.no_grad():
    acc = 0
    label_correct_rate = [0] * 20
    hit_rate = [0] * 20
    target_list = [109, 618, 536, 546, 347, 40, 415, 619, 208, 314, 8, 525, 714, 52, 221, 523, 785, 951, 984, 965]
    for i, (data, target, _) in enumerate(tqdm(test)):
        data, target = data.cuda(), target.cuda()
        
        output = client(data)
        output = server(output)
        output = output.argmax(dim=1)
        acc += (output == target).sum().item()
        # for i in target.cuda().cpu().numpy():
        #     if i not in target_list:
        #         target_list.append(i)
        for i in range(output.size(0)):
            for j in range(20):
                if target[i] == target_list[j]:
                    hit_rate[j] += 1
                    if output[i] == target[i]:
                        label_correct_rate[j] += 1

    print(len(test.dataset))
    print(acc / len(test.dataset))
    print([label_correct_rate[i] / max(1,hit_rate[i]) for i in range(20)])
    print(hit_rate)
    # print(target_list)

100%|██████████| 51/51 [00:19<00:00,  2.58it/s]

5087
0.8647532927068999
[0.968421052631579, 0.7032967032967034, 0.7459677419354839, 0.8801652892561983, 0.9766536964980544, 0.8007662835249042, 0.8226415094339623, 0.6859205776173285, 0.8445229681978799, 0.8941176470588236, 0.9090909090909091, 0.9473684210526315, 0.8909952606635071, 0.8111587982832618, 0.9267015706806283, 0.8371212121212122, 0.8869565217391304, 0.8619402985074627, 0.9875, 0.9484126984126984]
[285, 273, 248, 242, 257, 261, 265, 277, 283, 255, 286, 266, 211, 233, 191, 264, 230, 268, 240, 252]
